1. Dataset
2. Independent and dependent features
3. cleaning the data 1) Stemming 2) Stopwords
4. Fix a sentence length to fix the input
5. One hot representation
6. Embediing Layer
7. LSTM Neural Network

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
df=df.dropna()

In [7]:
df.shape

(18285, 5)

In [8]:
#Get the Dependent features
y = df['label']

In [9]:
X = df.drop('label',axis=1)

In [10]:
X.shape

(18285, 4)

In [11]:
import tensorflow as tf

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [13]:
## Vocabulary size
voc_size = 5000

OneHot Representation

In [14]:
messages = X.copy()

In [15]:
messages['title'][3]

'15 Civilians Killed In Single US Airstrike Have Been Identified'

In [16]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [17]:
import nltk
import re
from nltk.corpus import stopwords



In [18]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
corpus = []

for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'].iloc[i]) 
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)


In [19]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [21]:
onehot_rep = [one_hot(words,voc_size) for words in corpus]
onehot_rep


[[1458, 2257, 973, 797, 3505, 1829, 2937, 4367, 3981, 2755],
 [1450, 543, 2423, 135, 311, 4589, 114],
 [4499, 2395, 3379, 3249],
 [3871, 503, 4670, 2472, 1486, 3783],
 [3374, 311, 2070, 1119, 52, 188, 311, 1472, 2111, 1829],
 [2567,
  3694,
  3700,
  4372,
  3655,
  2140,
  3687,
  2444,
  4419,
  4953,
  4621,
  1541,
  2419,
  1711,
  114],
 [4649, 2242, 3264, 2562, 3765, 1577, 3292, 2453, 1419, 4930, 4102],
 [4083, 4241, 1647, 258, 3513, 2391, 2140, 3799, 1419, 4930, 4102],
 [43, 585, 367, 657, 4548, 4056, 2886, 4411, 2140, 2922],
 [4028, 3454, 3717, 3803, 1748, 1999, 2217, 3210],
 [4389, 1241, 2851, 1373, 411, 365, 967, 3641, 4158, 2771, 1614],
 [2472, 2166, 3505, 4056, 2140, 3513],
 [373, 1973, 2882, 3543, 4426, 4873, 2852, 3268, 500],
 [3074, 1344, 1609, 4700, 1071, 4568, 4211, 1419, 4930, 4102],
 [446, 20, 2187, 3601, 4761, 1419, 4930, 4102],
 [4140, 2463, 721, 2304, 4084, 2239, 2684, 4482, 603, 2322],
 [4256, 1864, 543],
 [837, 4533, 4567, 2816, 2140, 4807, 1778, 114],
 [4416, 

In [25]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [26]:
onehot_rep[1]

[1450, 543, 2423, 135, 311, 4589, 114]

Embedding Representation

In [30]:
sent_length = 20
embedded_docs = pad_sequences(onehot_rep,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4367 3981 2755]
 [   0    0    0 ...  311 4589  114]
 [   0    0    0 ... 2395 3379 3249]
 ...
 [   0    0    0 ... 1419 4930 4102]
 [   0    0    0 ... 3728  227 3983]
 [   0    0    0 ... 4798 4869 3494]]


In [32]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 1450,  543, 2423,  135,  311, 4589,  114])

In [38]:
#Creating Model
embedding_vector_feature=40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_feature,input_length=20))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.build(input_shape=(None, 50))
print(model.summary())

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ (None, 50, 40)              │         200,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 100)                 │          56,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 256,501 (1001.96 KB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

None


In [39]:
len(embedded_docs)

18285

In [40]:
y.shape

(18285,)

In [41]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [43]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [44]:
#Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.8031 - loss: 0.4213 - val_accuracy: 0.9079 - val_loss: 0.2153
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.9390 - loss: 0.1482 - val_accuracy: 0.9203 - val_loss: 0.2078
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.9655 - loss: 0.0976 - val_accuracy: 0.9171 - val_loss: 0.2564
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9742 - loss: 0.0703 - val_accuracy: 0.9163 - val_loss: 0.2524
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.9861 - loss: 0.0427 - val_accuracy: 0.9173 - val_loss: 0.3097
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9912 - loss: 0.0294 - val_accuracy: 0.9067 - val_loss: 0.3377
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.9926 - loss: 0.0213 - val_accuracy: 0.9099 - val_loss: 0.3835
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.9953 - loss: 0.0159 - val_accu

In [45]:
y_pred = model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [46]:
y_pred = np.where(y_pred>0.5,1,0)

In [47]:
from sklearn.metrics import confusion_matrix

In [48]:
confusion_matrix(y_test,y_pred)

array([[3101,  318],
       [ 223, 2393]], dtype=int64)

In [50]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.88      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

